In [ ]:
import json
import sys
from dotenv import load_dotenv, find_dotenv
import os

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

## Make a request to model without tools

In [ ]:
from aisuite import Client

client = Client()
# Configuring Azure. Rest all providers use environment variables for their parameters.
client.configure({"azure" : {
  "api_key": os.environ["AZURE_API_KEY"],
  "base_url": "https://aisuite-mistral-large-2407.westus3.models.ai.azure.com/v1/",
}})
# model = "anthropic:claude-3-5-sonnet-20241022"
# model = "aws:mistral.mistral-7b-instruct-v0:2"
# model = "azure:aisuite-mistral-large"
# model = "cohere:command-r-plus"
# model = "deepseek:deepseek-chat"
# model = "fireworks:accounts/fireworks/models/llama-v3p1-405b-instruct"
# model = "google:gemini-1.5-pro-002"
# model = "groq:llama-3.3-70b-versatile"
# model = "huggingface:meta-llama/Llama-3.1-8B-Instruct"
# model = "mistral:mistral-large-latest"
# model = "nebius:"
# model = "ollama:"
# model = "sambanova:Meta-Llama-3.3-70B-Instruct"
# model = "together:meta-llama/Llama-3.3-70B-Instruct-Turbo"
# model = "watsonx:"
model = "xai:grok-2-latest"

messages = [{
    "role": "user",
    "content": "What is the current temperature in San Francisco in Celsius?"}]

response = client.chat.completions.create(
    model=model, messages=messages)

print("For model: " + model)
print(response.choices[0].message.content)

## Equip model with tools

### Define the functions

In [ ]:
# Mock tool functions.
def get_current_temperature(location: str, unit: str):
    # Simulate fetching temperature from an API
    return {"temperature": 72}

def get_rain_probability(location: str):
    # Simulate fetching rain probability
    return {"location": location, "probability": 40}

# Function to get the available tools (functions) to provide to the model
# Note: we could use decorators or utils from OpenAI to generate this.
def get_available_tools():
    return [
        {   "type": "function",
            "function": {
                "name": "get_current_temperature",
                "description": "Get the current temperature for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g., San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["Celsius", "Fahrenheit"],
                            "description": "The temperature unit to use."
                        }
                    },
                    "required": ["location", "unit"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_rain_probability",
                "description": "Get the probability of rain for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g., San Francisco, CA"
                        }
                    },
                    "required": ["location"]
                }
            }
        }
    ]

In [ ]:
# Function to process tool calls and get the result
def handle_tool_call(tool_call):
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    # Map function names to actual tool function implementations
    tools_map = {
        "get_current_temperature": get_current_temperature,
        "get_rain_probability": get_rain_probability,
    }
    return tools_map[function_name](**arguments)

In [ ]:
# Function to format tool response as a message
def create_tool_response_message(tool_call, tool_result):
    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": tool_call.function.name,
        "content": json.dumps(tool_result)
    }

### Call the model with tools

In [ ]:
import json
import sys
from dotenv import load_dotenv, find_dotenv
import os

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

from aisuite import Client

client = Client()
client.configure({"azure" : {
  "api_key": os.environ["AZURE_API_KEY"],
  "base_url": "https://aisuite-mistral-large-2407.westus3.models.ai.azure.com/v1/",
}})

# model = "anthropic:claude-3-5-sonnet-20241022"
# model = "aws:mistral.mistral-7b-instruct-v0:2"
# model = "azure:aisuite-mistral-large"
# model = "cohere:command-r-plus"
# model = "deepseek:deepseek-chat"
# model = "fireworks:accounts/fireworks/models/llama-v3p1-405b-instruct"
# model = "google:gemini-1.5-pro-002"
# model = "groq:llama-3.3-70b-versatile"
# model = "huggingface:meta-llama/Llama-3.1-8B-Instruct"
# model = "mistral:mistral-large-latest"
# model = "nebius:"
# model = "ollama:"
# model = "sambanova:Meta-Llama-3.3-70B-Instruct"
# model = "together:meta-llama/Llama-3.3-70B-Instruct-Turbo"
# model = "watsonx:"
model = "xai:grok-2-latest"

messages = [{
    "role": "user",
    "content": "What is the current temperature in San Francisco in Celsius?"}]

tools = get_available_tools()

# Make the initial request to OpenAI API
response = client.chat.completions.create(
    model=model, messages=messages, tools=tools)

print(response)
print(response.choices[0].message)

### Process tool calls - Parse tool name, args, and call the function. Pass the result to the model.

In [ ]:
if response.choices[0].message.tool_calls:
    for tool_call in response.choices[0].message.tool_calls:
        tool_result = handle_tool_call(tool_call)
        print(tool_result)

        messages.append(response.choices[0].message) # Model's function call message
        messages.append(create_tool_response_message(tool_call, tool_result))
        # Send the tool response back to the model
        final_response = client.chat.completions.create(
            model=model, messages=messages, tools=tools)
        print(final_response.choices[0].message)
        
        # Output the final response from the model
        print(final_response.choices[0].message.content)